In [301]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [302]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [303]:
train = pd.read_csv('../data/prep/train3_train.csv')
test = pd.read_csv('../data/prep/test3_valid.csv')
submission = pd.read_csv('../data/prep/submission_valid.csv')

In [304]:
train

,id,Gr.Liv.Area,Garage.Cars,Garage.Area,Total.Bsmt.SF,X1st.Flr.SF,Full.Bath,Year.Built,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,total.Price.Index,target
0,1140,7.138073,1.098612,6.270988,7.138073,7.138073,1.098612,7.586804,3.713572,2.079442,3.713572,3.871201,12.556494,11.884496
1,1204,7.524561,1.098612,6.469250,6.803505,6.838405,1.098612,7.605890,1.098612,1.098612,1.098612,1.609438,12.716669,12.439875
2,228,7.482682,1.386294,6.790097,7.368340,7.482682,1.098612,7.580189,3.784190,3.988984,3.988984,4.672829,12.510173,11.608245
3,548,6.991177,1.098612,6.196444,6.991177,6.991177,0.693147,7.578145,3.135494,2.197225,4.060443,4.189655,12.035646,11.883806
4,325,7.284135,1.098612,6.018593,6.591674,6.591674,1.098612,7.603898,1.945910,1.945910,1.945910,2.564949,12.539143,12.149508
5,541,7.363280,1.386294,6.505784,7.239933,7.363280,1.098612,7.603399,2.079442,2.079442,2.079442,2.708050,12.989730,12.301342
6,1082,7.334329,0.693147,5.347108,6.629363,7.064759,0.693147,7.574558,4.174387,3.496508,4.174387,4.574711,11.790110,11.751950
7,33,6.985642,1.098612,6.184149,6.989335,6.985642,0.693147,7.592870,3.367296,3.367296,3.367296,4.043051,12.212616,11.805603
8,251,7.758333,1.386294,6.669498,7.120444,7.120444,1.098612,7.599401,2.772589,2.772589,2.772589,3.433987,13.194693,12.631344
9,743,6.880384,0.693147,5.723585,6.880384,6.880384,0.693147,7.578657,4.043051,4.043051,4.043051,4.727388,11.884117,11.715874


In [305]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [306]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [307]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [308]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [309]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [310]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [311]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2899561	test: 186937.2660165	best: 186937.2660165 (0)	total: 1.83ms	remaining: 5.49s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 186937.226
bestIteration = 76

Shrink model to first 77 iterations.
1 FOLD NMAE = 0.14841666718275215

2 FOLD Training.....
0:	learn: 0.2961609	test: 193017.1952342	best: 193017.1952342 (0)	total: 1.94ms	remaining: 5.82s
1000:	learn: 0.0634801	test: 193017.1023163	best: 193017.1023018 (999)	total: 1.85s	remaining: 3.7s
2000:	learn: 0.0544078	test: 193017.1019274	best: 193017.1017065 (1714)	total: 3.89s	remaining: 1.94s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 193017.1016
bestIteration = 2232

Shrink model to first 2233 iterations.
2 FOLD NMAE = 0.08061814663000456

3 FOLD Training.....
0:	learn: 0.2940162	test: 172422.4742863	best: 172422.4742863 (0)	total: 2.13ms	remaining: 6.38s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 172422.4743
bestIteration = 0

S

In [312]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.11279132864478612

2 FOLD Training.....
2 FOLD NMAE = 0.0890383683764395

3 FOLD Training.....
3 FOLD NMAE = 0.11131269541172087

4 FOLD Training.....
4 FOLD NMAE = 0.10217314444464946

5 FOLD Training.....
5 FOLD NMAE = 0.09276490785749981

6 FOLD Training.....
6 FOLD NMAE = 0.10739122322884233

7 FOLD Training.....
7 FOLD NMAE = 0.08826532171438746

8 FOLD Training.....
8 FOLD NMAE = 0.08937267602003393

9 FOLD Training.....
9 FOLD NMAE = 0.08394637846975296

10 FOLD Training.....
10 FOLD NMAE = 0.09745259426321724

10FOLD Mean of NMAE = 0.09745086384313298 & std = 0.009865706299794356


In [313]:
submission['target'] = ngb_pred

In [314]:
submission.to_csv("../out/ngb/ngb1_valid.csv", header = True, index = False)

In [315]:
submission['target'] = cb_pred

In [316]:
submission.to_csv("../out/cb/cb1_valid.csv", header = True, index = False)